## 出租车数据预处理

In [3]:
FILES_DIR = {
    # 文件夹
    "taxi_dir": "/Volumes/Age's SSD 1/SSD/数据/出租车/201401TRK/TRK20140101/",
    "taxi_sample": "/Volumes/Age's SSD 1/SSD/数据/出租车/201401TRK/TRK20140107/C03/浙C03860.LOG"
}

In [4]:
import pandas as pd
import json

In [5]:
from struct import *
import time

def read_taxi_raw_file(filename):
    fmt = '>ffQ'
    ps = []
    with open(filename, "rb") as f:
        line = f.read(39)
        while(line):
            s = line[0:16]
            # 读取二进制文件 https://docs.python.org/zh-cn/3/library/struct.html
            data = unpack(fmt, s)
            ps.append({
                "longitude": data[0],
                "latitude": data[1],
                "time": time.strftime("%Y-%m-%d %H:%M:%S",  time.localtime(data[2]/1000))
            })
            line = f.read(39)
    return ps

# 读取 Log 文件到 Dataframe
def read_taxi_df(filename):
    points = read_taxi_raw_file(filename)
    df_taxi = pd.DataFrame(points)
    return df_taxi

# read_taxi_df(FILES_DIR["taxi_sample"])

In [9]:
import json

def pack_one_taxi_to_json(df, carNo):
    points_str = df.to_json(orient='records')
    points = json.loads(points_str)
    taxi_trajectory = {
        "carNo": carNo,
        "points": points
    }
    return taxi_trajectory
    # file_name = carNo + '.json'
# j = pack_one_taxi_to_json(df_taxi_points, carNo)
# print(json.dumps(j, indent=4))

def write_json_to_file(json_obj, filename='taxi.json'):
    path = '../data_in_use/'
    filename = path + filename
    with open(filename, 'w') as outfile:
        json.dump(json_obj, outfile)

In [10]:
from os import walk

# 获取文件列表
def list_files():
    raw_files = []
    BASE_PATH = FILES_DIR["taxi_dir"]
    for (dirpath, dirnames, filenames) in walk(FILES_DIR["taxi_dir"]):
        if(len(dirnames) == 0):
            for filename in filenames:
                raw_files.append(
                    [dirpath + '/' + filename, filename]
                )
    return raw_files

# 读取原始文件 并转换至 json 格式
def read_files(num = 20):
    files = list_files()
    jsons = []
    c = 0
    for [f, carNo] in files:
        print('doing ' + carNo + '...')
        c = c + 1
        df = read_taxi_df(f)
        taxi_json = pack_one_taxi_to_json(df, carNo)
        jsons.append(taxi_json)
        if c >= num:
            break
    write_json_to_file(jsons)
    print('done !')
read_files()

doing 浙C02668.LOG...
doing 浙C02678.LOG...
doing 浙C03860.LOG...
doing 浙C04248.LOG...
doing 浙C04337.LOG...
doing 浙C04348.LOG...
doing 浙C04351.LOG...
doing 浙C04628.LOG...
doing 浙C04651.LOG...
doing 浙C04652.LOG...
doing 浙C04658.LOG...
doing 浙C04675.LOG...
doing 浙C04876.LOG...
doing 浙C04937.LOG...
doing 浙C04974.LOG...
doing 浙C05132.LOG...
doing 浙C05135.LOG...
doing 浙C05196.LOG...
doing 浙C05202.LOG...
doing 浙C05203.LOG...
done !
